In [8]:
%%capture
! pip install opencv-python numpy pytesseract PaddleOCR easyocr


In [9]:
import cv2
import numpy as np
import pytesseract

In [10]:
image = cv2.imread('images/receipt_image.jpg', 0)
color = cv2.imread('images/receipt_image.jpg')
hsv = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)

hue, saturation, value = cv2.split(hsv)

In [11]:
def show(image):
    
    resized_height = 720
    percent = resized_height / len(image)
    resized_width = int(percent * len(image[0]))
    gray = cv2.resize(image,(resized_width,resized_height))

    cv2.imshow('cringe', gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [12]:
if np.mean(color) < 128:
    thresh = cv2.threshold(hue, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
else:
    thresh = cv2.threshold(hue, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [13]:
eroded = cv2.erode(thresh, kernel=None, iterations=7)
dilated = cv2.dilate(eroded, kernel=None, iterations=15)
eroded = cv2.erode(dilated, kernel=None, iterations=100)
dilated = cv2.dilate(eroded, kernel=None, iterations=100)

In [14]:
show(dilated)

In [15]:
def extract_largest_rectangular_contour(color_image, preprocessed_image):
    contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = None
    largest_area = 0
    largest_approx = None
    
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if len(approx) == 4:
            area = cv2.contourArea(contour)
            if area > largest_area:
                largest_area = area
                largest_contour = contour
                largest_approx = approx
                
    if largest_contour is not None:
        cv2.drawContours(color_image, [largest_approx], -1, (0, 255, 0), 3)
        # x, y, w, h = cv2.boundingRect(largest_contour)
        # cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        return color_image, largest_approx
    
    return color_image, None

In [16]:
im, cor = extract_largest_rectangular_contour(color, dilated)
show(im)

In [17]:
top = max(cor[0][0][1], cor[3][0][1])
bottom = min(cor[1][0][1], cor[2][0][1])
left = max(cor[0][0][0], cor[1][0][0])
right = min(cor[2][0][0], cor[3][0][0])
cropped = image[top:bottom, left:right]
show(cropped)

In [18]:
def order_points(coords):
    pts = coords - [left, top]
    pts = pts.reshape(4, 2)
    sum_pts = pts.sum(axis=1)
    diff_pts = np.diff(pts, axis=1)
    
    top_left = pts[np.argmin(sum_pts)]
    top_right = pts[np.argmin(diff_pts)]
    bottom_right = pts[np.argmax(sum_pts)]
    bottom_left = pts[np.argmax(diff_pts)]
    
    return np.array([top_left, top_right, bottom_right, bottom_left])

def calculate_size(img, ordered):
    width = int(img.shape[1] * 0.9)
    aspect_ratio = np.linalg.norm(ordered[0] - ordered[3]) / np.linalg.norm(ordered[0] - ordered[1])
    return width, int(width * aspect_ratio)

def process_image(img, contour):
    ordered = order_points(contour)
    img_with_pts = img.copy()
    for pt in ordered:
        img_with_pts = cv2.circle(img_with_pts, tuple(pt), 10, (255, 255, 255), -1)
  
    w, h = calculate_size(img, ordered)
    
    pts1 = np.float32(ordered)
    pts2 = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    
    corrected_img = cv2.warpPerspective(img, matrix, (w, h))
    
    return img_with_pts, corrected_img

In [19]:
_, corrected = process_image(cropped, cor)
show(corrected)

In [20]:
# clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
# clahe_img = clahe.apply(corrected)
# show(clahe_img)

In [21]:
def unsharp_masking(image, k=1):
    blur = cv2.GaussianBlur(image, (11,11), 0)
    return cv2.addWeighted(image, k+1, blur, -k, 1)

In [22]:
unsharp = unsharp_masking(corrected, 11)
show(unsharp)

In [23]:
thresh = np.where(unsharp > 64, 255, 0).astype(np.uint8)
show(thresh)

In [24]:
text = pytesseract.image_to_string(thresh)
sentences=text.split('\n')
for s in sentences:
    print(s)

BACCHUS INN”
far A Restaurant
Coatnsgar.hantoa!
= CASH HEN ==
Sones att wo, : SESS
Hetoe T5

Oty Rate, anount

FINGER CHIPS, 1
CH] GHEE ROAST BONLESS
Fue

CHI SHEE ROAST BONLESS
HALE

eRISPY CORN
ORAGON CH

ANAL RAWA FRY
ARDARA KARAS

SOFT DRUIKS SOOM.
SERVICE CaBRE
BOTTLE WATER 1 UR

2230.00

TEACHERS 69 HL PET 285

1

FC LUYURY 30H. 7 6 © 420
BLACK DOG SOM 3150480
BUWISER MASNEM 30M. 7-200 1400
PCMONGY YOGHA 20H. 1B GOS.
ADVAN CHIPS. to 20
BAISHY SOUR 10 0

VAS/3

cstinsaerureeagonis

aPricg inclusive of All Taxesé
EOE, Thank You Vistt‘Avain

WT

 



In [25]:
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='en')  

result = ocr.ocr(thresh, cls=True)

for line in result[0]:
    print(f"Detected text: {line[1][0]} with confidence: {line[1][1]}")

ModuleNotFoundError: No module named 'paddleocr'

In [ ]:
import easyocr

reader = easyocr.Reader(['en'])  

result = reader.readtext(thresh)

for detection in result:
    text = detection[1]
    confidence = detection[2]
    print(f"Detected text: {text} with confidence: {confidence}")

Detected text: BACCHUS with confidence: 0.9886242994701175
Detected text: INN with confidence: 0.8623996734977019
Detected text: Mr with confidence: 0.12001676718911852
Detected text: Festauram with confidence: 0.6476787484592748
Detected text: Laxinoxr Honlpal with confidence: 0.13554649467996352
Detected text: Casy hin with confidence: 0.025907343226557075
Detected text: G&; with confidence: 0.11120615107758094
Detected text: 88/09r24 with confidence: 0.4278346884374021
Detected text: B with confidence: 0.10824091905023181
Detected text: m. with confidence: 0.28639958028833695
Detected text: S951 with confidence: 0.08385501056909561
Detected text: No . with confidence: 0.2053610029537157
Detected text: Itez with confidence: 0.5512901544570923
Detected text: Oty Rate with confidence: 0.7404747270779826
Detected text: Arurt with confidence: 0.7611616582955898
Detected text: FIKGER CXIp5 with confidence: 0.3765034395559343
Detected text: CHI GZE RJRST EOXLesS with confidence: 0.09115755